In [2]:
import requests
import zipfile
from bs4 import BeautifulSoup
import pandas as pd
import os
import warnings
from  dask import dataframe as dd
import shutil
warnings.filterwarnings("ignore")

In [3]:

url = 'https://dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_operadora/'
dir = 'Cancelamento'

In [3]:
#Extrair SIB Inativos

def extrair_links(url):
    reponse = requests.get(url)
    
    if reponse.status_code == 200:
        soup = BeautifulSoup(reponse.text, 'html.parser')
        
        links = soup.find_all('td') 

        links_ext = []
        for link in links:
            a_tag = link.find('a')
            if a_tag and 'href' in a_tag.attrs:
                href = a_tag['href']
                if href.startswith('sib_inativo'):
                    links_ext.append(href)

        return links_ext
    else:
        print("Erro ao acessar a página:", response.status_code)
        return None

links = extrair_links(url)

In [4]:
#create urls
url_md = [url + i for i in links]

In [5]:
os.makedirs(dir, exist_ok=True)

for i, zip_url in enumerate(url_md):
    response = requests.get(zip_url)

    if response.status_code == 200:
        zip_filename = links[i]  
        zip_path = os.path.join(dir, zip_filename)

        with open(zip_path, 'wb') as file:
            file.write(response.content)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dir)

        # Remove zip file
        os.remove(zip_path)
        print(f"{zip_filename} extraído e removido com sucesso.")
    else:
        print('Não foi possível baixar o arquivo .zip. Status code:', response.status_code)


sib_inativo_AC.zip extraído e removido com sucesso.
sib_inativo_AL.zip extraído e removido com sucesso.
sib_inativo_AM.zip extraído e removido com sucesso.
sib_inativo_AP.zip extraído e removido com sucesso.
sib_inativo_BA.zip extraído e removido com sucesso.
sib_inativo_CE.zip extraído e removido com sucesso.
sib_inativo_DF.zip extraído e removido com sucesso.
sib_inativo_ES.zip extraído e removido com sucesso.
sib_inativo_GO.zip extraído e removido com sucesso.
sib_inativo_MA.zip extraído e removido com sucesso.
sib_inativo_MG.zip extraído e removido com sucesso.
sib_inativo_MS.zip extraído e removido com sucesso.
sib_inativo_MT.zip extraído e removido com sucesso.
sib_inativo_PA.zip extraído e removido com sucesso.
sib_inativo_PB.zip extraído e removido com sucesso.
sib_inativo_PE.zip extraído e removido com sucesso.
sib_inativo_PI.zip extraído e removido com sucesso.
sib_inativo_PR.zip extraído e removido com sucesso.
sib_inativo_RJ.zip extraído e removido com sucesso.
sib_inativo_

In [4]:
dtypes = {
    'DT_NASCIMENTO': object,
    'DT_REATIVACAO': object,
    'DT_ULTIMA_REATIVACAO': object,
    'DT_ULTIMA_MUDA_CONTRATUAL':object,
    'NR_PLANO_SCPA': object,
    'LG_RESIDE_EXTERIOR': object,
    'ID_MOTIVO_MOVM_CANCELAMENTO':object,
    'LG_COBERTURA_PARCIAL_TEMP': object,
    'ID_TIPO_DEPENDENTE': object,
    'LG_ITEM_EXCLUIDO_COBERTURA': object,
    'NR_PLANO_RPS': object,
    'ID_MOTIVO_MOVM_INCLUSAO':object,
    'ID_MOTIVO_MOVIMENTO': object,
    'CD_PLANO_SCPA':object,
    'ID_BENE_TIPO_DEPENDENTE':object,
    'LG_ITENS_EXCLUIDO_COBERTURA':object,
    'ID_BENE_MOTIV_CANCELAMENTO':object,
    'CD_PLANO_RPS': object,
    'CD_BENE_MOTV_INCLUSAO': object
    
}

In [5]:
dfs_filtrados = []
for arquivo in os.listdir(dir):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(dir, arquivo)

        # Lê com Dask
        try:
            df = dd.read_csv(caminho_arquivo, sep=";", encoding='ISO-8859-1', dtype= dtypes)

            # Filtra os anos 2024 e 2025
            df_filtrado = df[df['DT_ULTIMO_CANCELAMENTO'].str.contains('^2025-|^2024-', regex=True, na=False)]

            dfs_filtrados.append(df_filtrado)

            # Exclui o arquivo após ler e filtrar
            #os.remove(caminho_arquivo)
            #print(f'{arquivo} processado e excluído.')
        
        except Exception as e:
            print(f'Erro ao processar {arquivo}: {e}')



In [6]:
# Consolida tudo em um DataFrame Dask
if dfs_filtrados:
    df_consolidado = dd.concat(dfs_filtrados)
    op = [5711,327417,335100,350249,366561,368253]
    df_consolidado = df_consolidado[df_consolidado['REGISTRO_OPERADORA'].isin(op)]

    # Opcional: converter para pandas se for pequeno o suficiente
    df_resultado = df_consolidado.compute()
    
    # Salva em um novo arquivo CSV consolidado
    df_resultado.to_csv('consolidado_2024_2025.csv', index=False)
    print("Arquivo consolidado salvo com sucesso.")
else:
    print("Nenhum dado correspondente encontrado.")

Arquivo consolidado salvo com sucesso.
